## Gaussian Process Emulation

Gaussian process emulation has been in use since 1980s as a well-known non-parametric Bayesian method. It builds *cheap-to-run* emulators for *expensive-to-run* simulation models (referred to as simulators) so as to enable analyses requiring a large number of simulation runs.

Given a simulator $y=f(x)$ with a $p$-dimensional input $x \in \mathbb{R}^{p}$ and scalar output $y \in \mathbb{R}$, the function $f(\cdot)$ is treated as an unknown function. The prior belief of $f(\cdot)$ is modeled by a Gaussian process, namely

$$
\begin{equation}
    f(\cdot)\sim \mathcal{GP}(m(\cdot;\beta),C(\cdot;\cdot;\sigma^{2},\gamma))
\end{equation}
$$

The Gaussian process is fully defined by its mean function $m(\cdot;\beta)$ and covariance function $C(\cdot;\cdot;\sigma^2,\gamma)$, where $\beta$, $\sigma^2$, $\gamma$ are unknown hyperparameters. This Gaussian process prior is then updated to a Gaussian process posterior based on input-output data $D$ of a modest number of simulation runs, known as training data. The output of the function at any new input can then be predicted almost instantaneously by

$$
\begin{equation}
    y^*|D,\beta,\sigma^2,\gamma \sim N(m´,\sigma^{2}c´)
\end{equation}
$$

where $m´$ and $c´$ are updated mean function and covariance function respectively. There are different ways to deal with the unknown hyperparameters $\beta$, $\sigma^2$, $\gamma$, varying from non-Bayesian to fully Bayesian approaches. In this paper, we use the partially Bayesian approach which first integrates out $\beta$, $\sigma^2$ following the Bayesian way and then plugs in the maximum marginal posterior estimation of $\gamma$ in a non-Bayesian way. In this case, the posterior given in Eq. \ref{eq:eq2} is no longer a Gaussian process but a Student’s-t process, see \citep{zhao2020emulatorbased} for detailed description.
For multi-output simulators, such as the landslide run-out model, multivariate Gaussian process emulation is required. In this study, we use the parallel partial Gaussian process emulator developed by \citep{Gu_Berger_2016}. Both the maximum marginal posterior estimation of $\gamma$ and the parallel partial Gaussian process emulator have been implemented in the R package RobustGaSP, which was presented by \citep{Gu_et_al_2019}. We combine it with the landslide run-out solver r.avaflow v2.3 in this study.

We will have a look at the [RobustGaSP](https://cran.r-project.org/web/packages/RobustGaSP/RobustGaSP.pdf) package by maintained by [Mengyang Gu](mailto:mengyang@pstat.ucsb.edu).

In [1]:
library(RobustGaSP)
library(lhs)
library(scatterplot3d)
# dimensional of the inputs
dim_inputs <- 3
# number of the inputs
num_obs <- 30
# uniform samples of design
input <- maximinLHS(n = num_obs, k = dim_inputs) ##maximin lhd sample

Warning message:
"package 'RobustGaSP' was built under R version 3.6.3"#########
##
## Robust Gaussian Stochastic Process, RobustGaSP Package
## Copyright (C) 2016-2022 Mengyang Gu, Jesus Palomo and James O. Berger
#########

Attaching package: 'RobustGaSP'

The following object is masked from 'package:stats':

    simulate

Warning message:
"package 'lhs' was built under R version 3.6.3"

We wil generate output data using the curved function given in [Dette and Pepelyshev (2010)](https://doi.org/10.1198/TECH.2010.09157).

$$
f(x) = 4 (x_1 - 2 + 8x_2 - 8{x_2}^2)^2 + (3 - 4x_2)^2 + 16 \sqrt{x_3 + 1}{(2x_3 - 1)}^2
$$

The function is evaluated on the cube for $x_1$, $x_2$ and $x_3$ between 0 and 1. We create an input data frame using the latin hypercube sampling method.

In [2]:
input <- maximinLHS(n = 100, k = 3)
colnames(input) <- c("x1", "x2", "x3")
head(input)

x1,x2,x3
0.6137628,0.1094040,0.5207374
0.7856541,0.1355144,0.3154205
0.5284009,0.8760465,0.8016871
0.2915581,0.5301255,0.3443643
0.5683302,0.2846974,0.7829776
0.4409170,0.4294738,0.8105108


In [3]:
dette_and_pepelyshev <- function(x1, x2, x3) {
    term1 <- 4 * ((x1 - 2 + 8*x2 - 8 * (x2^2))^2)
    term2 <- (3 - 4 * x2)^2
    term3 <- 16 * sqrt(x3 + 1) * (2 * x3 - 1)^2
    result <- term1 + term2 + term3
    return(result)
}
output <- dette_and_pepelyshev(input[, 1], input[, 2], input[, 3])

In [5]:
m1<- rgasp(design = input, response = output, lower_bound=FALSE)
num_testing_input <- 1000
# generate points to be predicted
testing_input <- matrix(runif(num_testing_input*dim_inputs),num_testing_input,dim_inputs)
# Perform prediction
m1.predict<-predict(m1, testing_input, outasS3 = FALSE)
# Predictive mean
#m1.predict@mean
# The following tests how good the prediction is
testing_output <- matrix(0,num_testing_input,1)
for(i in 1:num_testing_input){
testing_output[i]<-dettepepel.3.data(testing_input[i,])
}
# compute the MSE, average coverage and average length
# out of sample MSE
MSE_emulator <- sum((m1.predict@mean-testing_output)^2)/(num_testing_input)
# proportion covered by 95% posterior predictive credible interval
prop_emulator <- length(which((m1.predict@lower95<=testing_output)
&(m1.predict@upper95>=testing_output)))/num_testing_input
# average length of posterior predictive credible interval
length_emulator <- sum(m1.predict@upper95-m1.predict@lower95)/num_testing_input

The upper bounds of the range parameters are Inf Inf Inf 
The initial values of range parameters are 0.009199126 0.009196666 0.009200836 
Start of the optimization  1  : 
The number of iterations is  41 
 The value of the  marginal posterior  function is  36.27718 
 Optimized range parameters are 42.09951 13.1401 40.64215 
 Optimized nugget parameter is 0 
 Convergence:  FALSE 
The initial values of range parameters are 0.2752337 0.275139 0.2752996 
Start of the optimization  2  : 
The number of iterations is  23 
 The value of the  marginal posterior  function is  36.25247 
 Optimized range parameters are 42.75381 13.35304 41.42685 
 Optimized nugget parameter is 0 
 Convergence:  TRUE 


You can watch an example of how to use Gaussian Process Emulation in emulating the output of landslide run-out simulations.

<div align="center">
  <a href="https://www.youtube.com/watch?v=BLdE9f5XOVM"><img src="https://img.youtube.com/vi/BLdE9f5XOVM/0.jpg" alt="IMAGE ALT TEXT"></a>
</div>